In [1]:
# The intent of the notebook is to begin modelling the baseball data. This will likely not be the final model, 
#     however, it will act as a building block to more advanced models.
# Version: 1.0

In [23]:
import import_ipynb
from sklearn import svm
from pymysql import connect
from pathlib import Path
path.append('../../../') 
from BaseballAnalytics.bin.app_utils.queries import Queries
import numpy

In [19]:
# Connect to the database.
conn = connect(host="localhost", user="root", passwd="praquplDop#odlg73h?c", db="baseball_stats_db")
qu = Queries(conn)

In [24]:
# Get all the game ids. The information is returned as the Game Id, Year, Date, an Month.
all_game_ids = qu.get_all_game_ids()         
all_game_ids[0:5]

(('TEX199004090', 1990, 9, 4),
 ('SLN199004090', 1990, 9, 4),
 ('OAK199004090', 1990, 9, 4),
 ('CAL199004090', 1990, 9, 4),
 ('CHA199004090', 1990, 9, 4))

In [7]:
players_in_game = """
select Distinct game_day.Game_ID, batter_in_event.Batting_Team, pitcher_in_event.Pitcher_Name,       # Home Team equals One for Batting Team
event_catcher.Catcher, event_centre_field.Center_Field, event_left_field.Left_Field,
event_right_field.Right_Field, event_first_base.First_Base, event_second_base.Second_Base, 
event_third_base.Third_Base, event_shortstop.Shortstop
    from game_day
	inner join event_instance on game_day.Game_ID=event_instance.Game_ID
	inner join batter_in_event on event_instance.idEvent=batter_in_event.idEvent
    inner join pitcher_in_event on pitcher_in_event.idEvent=event_instance.idEvent
    inner join event_catcher on event_catcher.idEvent=event_instance.idEvent
    inner join event_centre_field on event_centre_field.idEvent=event_instance.idEvent
    inner join event_left_field on event_left_field.idEvent=event_instance.idEvent
    inner join event_right_field on event_right_field.idEvent=event_instance.idEvent
    inner join event_first_base on event_first_base.idEvent=event_instance.idEvent
    inner join event_second_base on event_second_base.idEvent=event_instance.idEvent
    inner join event_third_base on event_third_base.idEvent=event_instance.idEvent
    inner join event_shortstop on event_shortstop.idEvent=event_instance.idEvent
    where event_instance.Game_ID = '{}' order by batter_in_event.batting_team, CHAR_LENGTH(event_instance.idEvent), event_instance.idEvent;
    """.format(game_id)

NameError: name 'game_id' is not defined